In [2]:
from langchain_community.document_loaders import DirectoryLoader


In [3]:
markdown_path = "/home/pradanaend/Documents/Project/chatbot-bank/data/processed"
loader = DirectoryLoader(markdown_path)

data = loader.load()
readme_content = data[0].page_content


In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def split_docs(documents,chunk_size=1000,chunk_overlap=500):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
  docs = text_splitter.split_documents(documents)
  return docs

docs = split_docs(data)

In [5]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

/home/pradanaend/Documents/Project/chatbot-bank/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from langchain.vectorstores import Chroma
db = Chroma.from_documents(docs, embeddings, persist_directory="./chroma_db")

In [7]:
# Load from disk
vectorstore_disk = Chroma(
                        persist_directory="./chroma_db",       # Directory of db
                        embedding_function=embeddings   # Embedding model
                   )
# Get the Retriever interface for the store to use later.
# When an unstructured query is given to a retriever it will return documents.
# Read more about retrievers in the following link.
# https://python.langchain.com/docs/modules/data_connection/retrievers/
#
# Since only 1 document is stored in the Chroma vector store, search_kwargs `k`
# is set to 1 to decrease the `k` value of chroma's similarity search from 4 to
# 1. If you don't pass this value, you will get a warning.
retriever = vectorstore_disk.as_retriever(search_kwargs={"k": 11})


/tmp/ipykernel_392791/1452965151.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore_disk = Chroma(


In [8]:
query = """"
BNI Life Spectra Double Power
"""
matching_docs = db.similarity_search(query, k=10)

In [9]:
matching_docs

[Document(metadata={'source': '/home/pradanaend/Documents/Project/chatbot-bank/data/processed/RIPLAY_Umum_BNI_Life_Spectra_Double_Power.md'}, page_content='RINGKASAN INFORMASI PRODUK DAN LAYANAN (RIPLAY) VERSI UMUM\n\nPenanggung PT BNI Life Insurance Deskripsi Produk BNI Life Spectra Double Power merupakan Produk Asuransi yang memberikan gabungan manfaat Endowment (manfaat meninggal dunia dan manfaat hidup) serta nilai tunai. Pemegang Polis Perorangan Tertanggung Perorangan Nama Produk BNI Life Spectra Double Power Jenis Produk Dwiguna Kombinasi Mata Uang Rupiah (IDR)\n\nBNI LIFE SPECTRA DOUBLE POWER\n\nFitur Utama Asuransi\n\nMasa Asuransi Uang Pertanggungan 5 tahun Maksimal 105% dari Premi Total Sekaligus 10 tahun Maksimal 110% dari Premi Total Sekaligus 15 tahun Maksimal 120% dari Premi Total Sekaligus 20 tahun Maksimal 130% dari Premi Total Sekaligus'),
 Document(metadata={'source': '/home/pradanaend/Documents/Project/chatbot-bank/data/processed/RIPLAY_Umum_BNI_Life_Spectra_Double_

In [10]:

from langchain_google_genai import ChatGoogleGenerativeAI

# If there is no environment variable set for the API key, you can pass the API
# key to the parameter `google_api_key` of the `ChatGoogleGenerativeAI` function:
# `google_api_key="key"`.
llm = ChatGoogleGenerativeAI(model="gemini-pro",
                 temperature=0.7, top_p=0.85)

In [11]:
from langchain import PromptTemplate
from langchain import hub
from langchain.docstore.document import Document
from langchain.document_loaders import WebBaseLoader
from langchain.schema import StrOutputParser
from langchain.schema.prompt_template import format_document
from langchain.schema.runnable import RunnablePassthrough

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [12]:
from langchain_google_genai import ChatGoogleGenerativeAI

# If there is no environment variable set for the API key, you can pass the API
# key to the parameter `google_api_key` of the `ChatGoogleGenerativeAI` function:
# `google_api_key="key"`.
llm = ChatGoogleGenerativeAI(model="gemini-pro",
                 temperature=0.7, top_p=0.85)
     

In [42]:
# Prompt template to query Gemini
llm_prompt_template = """You are an assistant for question-answering tasks.
Use the following context to answer the question.
Use five sentences maximum and keep the answer concise.\n
Ensure to provide proper reference.
Ensure to search from all document provide.
If you don't find the answer, ask to contact the office.
Ensure to use Bahasa Indonesia.
Question: {question} \nContext: {context} \nAnswer:"""

llm_prompt = PromptTemplate.from_template(llm_prompt_template)

print(llm_prompt)

input_variables=['context', 'question'] input_types={} partial_variables={} template="You are an assistant for question-answering tasks.\nUse the following context to answer the question.\nUse five sentences maximum and keep the answer concise.\n\nEnsure to provide proper reference.\nEnsure to search from all document provide.\nIf you don't find the answer, ask to contact the office.\nEnsure to use Bahasa Indonesia.\nQuestion: {question} \nContext: {context} \nAnswer:"


In [43]:
# Combine data from documents to readable string format.
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Create stuff documents chain using LCEL.
#
# This is called a chain because you are chaining together different elements
# with the LLM. In the following example, to create the stuff chain, you will
# combine the relevant context from the website data matching the question, the
# LLM model, and the output parser together like a chain using LCEL.
#
# The chain implements the following pipeline:
# 1. Extract the website data relevant to the question from the Chroma
#    vector store and save it to the variable `context`.
# 2. `RunnablePassthrough` option to provide `question` when invoking
#    the chain.
# 3. The `context` and `question` are then passed to the prompt where they
#    are populated in the respective variables.
# 4. This prompt is then passed to the LLM (`gemini-pro`).
# 5. Output from the LLM is passed through an output parser
#    to structure the model's response.
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | llm_prompt
    | llm
    | StrOutputParser()
)

In [45]:
rag_chain.invoke(
    """Apakah saya bisa melakukan penari-kan  Nilai  Tunai  dari  polis  asuransi  saya?"""
)
    

'Ya, Anda dapat melakukan penarikan Nilai Tunai dari polis asuransi Anda dengan mengajukan Pinjaman Nilai Tunai. Persyaratannya antara lain:\n- Dana maksimal yang dapat dipinjam adalah 80% dari total nilai tunai\n- Mengisi Formulir pengajuan pinjaman polis\n- Melampirkan fotokopi KTP/identitas diri yang berlaku\n- Mengisi Surat Kuasa jika nomor rekening penerima manfaat berbeda dengan nomor rekening yang tercantum di polis.\n\n(Referensi: Dokumen yang disediakan, Q&A 1)'